In [ ]:
import os, shutil
train_dir = 'train'
validation_dir = 'validation'
test_dir = 'test'
from keras.utils import image_dataset_from_directory
IMG_SIZE = 150
train_dataset = image_dataset_from_directory(train_dir,image_size=(IMG_SIZE, IMG_SIZE),batch_size=32,label_mode='categorical')
validation_dataset = image_dataset_from_directory(validation_dir,image_size=(IMG_SIZE, IMG_SIZE),batch_size=32,label_mode='categorical')
test_dataset = image_dataset_from_directory(test_dir,image_size=(IMG_SIZE, IMG_SIZE),batch_size=32,label_mode='categorical')

In [ ]:
from keras.applications.vgg16 import VGG16
conv_base = VGG16(weights="imagenet", include_top=False)
conv_base.trainable = False

In [ ]:
train_features, train_labels = get_features_and_labels(train_dataset)
val_features, val_labels = get_features_and_labels(validation_dataset)
test_features, test_labels = get_features_and_labels(test_dataset)

In [ ]:
from tensorflow import keras
import numpy as np
def get_features_and_labels(dataset):
  all_features = []
  all_labels = []
  for images, labels in dataset:
    preprocessed_images = keras.applications.vgg16.preprocess_input(images)
    features = conv_base.predict(preprocessed_images)
    all_features.append(features)
    all_labels.append(labels)
  return np.concatenate(all_features), np.concatenate(all_labels)

In [ ]:
from tensorflow import keras
from keras import layers
from keras import models

inputs = keras.Input(shape=(4, 4, 512))
x = layers.Flatten()(inputs)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(10, activation="softmax")(x)
model = keras.Model(inputs, outputs)

In [ ]:
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

In [ ]:
from tensorflow import keras
callbacks_list = [
    keras.callbacks.EarlyStopping(monitor="val_loss",patience=10)
    #keras.callbacks.ModelCheckpoint(filepath='model_tl_adam.keras',save_best_only=True,monitor='val_acc',verbose=1)
]

In [ ]:
history = model.fit(train_features,train_labels,epochs=20,validation_data=(val_features,val_labels),callbacks=callbacks_list)

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
from keras import models
inputs = keras.Input(shape=(150, 150, 3))
x = keras.applications.vgg16.preprocess_input(inputs)
x = conv_base(x)
outputs = model(x)
full_model = keras.Model(inputs, outputs)

In [ ]:
full_model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

In [ ]:
full_model.save('TL_without_data_augmentation.h5')

In [ ]:
from tensorflow import keras
loaded_model = keras.models.load_model('TL_without_data_augmentation.h5')
val_loss, val_acc = loaded_model.evaluate(validation_dataset)
print('val_acc:', val_acc)